In [73]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from datetime import date
from datetime import time
from sklearn.cluster import KMeans

from datetime import datetime

In [2]:
order_detail_df = pd.read_excel("data/order.xlsx", parse_dates = ['Ngày phát sinh'])
user_df = pd.read_excel("data/user.xlsx")
listening_detail_df = pd.read_excel("data/listening.xlsx")

In [16]:
# Chuyển User_ID và Order_ID, Playlist ID sang category
order_detail_df['Order ID'] = order_detail_df['Order ID'].astype('category')
order_detail_df['ID của user'] = order_detail_df['ID của user'].astype('category')

user_df['User ID'] = user_df['User ID'].astype('category')

listening_detail_df['Playlist ID']=listening_detail_df['Playlist ID'].astype('category')
listening_detail_df['User ID']=listening_detail_df['User ID'].astype('category')

# usage_df['User ID']=usage_df['User ID'].astype('category')

In [18]:
# Chuyển Ngày phát sinh lượt nghe sang kiểu dữ liệu date
listening_detail_df["Ngày phát sinh lượt nghe"] = listening_detail_df["Ngày phát sinh lượt nghe"].map(
                                        lambda x:date(int(x.split("-")[2]),int(x.split("-")[1]),int(x.split("-")[0])))
listening_detail_df["Ngày phát sinh lượt nghe"] = listening_detail_df["Ngày phát sinh lượt nghe"].astype("datetime64")

In [64]:
order_summary_df = order_detail_df.groupby('ID của user').agg({'Ngày phát sinh': ['min',"max"]}).reset_index()
order_summary_df.columns = ['User_ID','min_purchase_date',"max_purchase_date"]

In [54]:
listening_summary_df = listening_detail_df.groupby('User ID').agg({"Ngày phát sinh lượt nghe":['min','max','count']}).reset_index()
listening_summary_df.columns = ['User_ID','first_listen_date','last_listen_date','listen_cnt']

### Cắt nhỏ giá trị ngày nghe

In [78]:
##### Cắt nhỏ giá trị order_date

listening_detail_df['listen_year'] = listening_detail_df['Ngày phát sinh lượt nghe'].map(lambda x: x.strftime('%Y'))
listening_detail_df['listen_month'] = listening_detail_df['Ngày phát sinh lượt nghe'].map(lambda x: x.strftime('%m'))
listening_detail_df['listen_day'] = listening_detail_df['Ngày phát sinh lượt nghe'].map(lambda x: x.strftime('%d'))

In [87]:
listening_detail_df['listen_month_year'] = listening_detail_df['Ngày phát sinh lượt nghe'].map(lambda x: x.strftime('%m/%Y'))

### Calculate những dữ liệu để phân tích ROLLRATE

In [65]:
# Chỉ xem những users có nghe và orders
# Lấy user_id từ 2 bảng order_summary_df và listening_summary_df gộp vào 1 bảng
user_cluster_df= pd.DataFrame(pd.concat(objs=[listening_summary_df['User_ID'],order_summary_df['User_ID']]).drop_duplicates().reset_index(drop=True))
user_cluster_df.columns = ["User_ID"]
user_cluster_df.shape[0]

27458

In [66]:
## Lấy ngày signup date
user_cluster_df = user_cluster_df.join(user_df[["User ID","Ngày đăng ký tài khoản"]].set_index("User ID"),
                                       on="User_ID",how="inner")
user_cluster_df = user_cluster_df.rename(columns = {'Ngày đăng ký tài khoản':'signup_date'})

# Tất cả các user đều có ngày sign up

In [67]:
## Lấy ngày đổi trạng thái

user_cluster_df = user_cluster_df.join(other=order_summary_df[["User_ID","min_purchase_date","max_purchase_date"]].set_index('User_ID'),how='left',on='User_ID')

In [68]:
## Lấy ngày nghe đầu tiên và cuối cùng
user_cluster_df= user_cluster_df.join(other=listening_summary_df[["User_ID","first_listen_date","last_listen_date"]].set_index('User_ID'),how='left',on='User_ID')

In [69]:
user_cluster_df.head(1)

,User_ID,signup_date,change_status_date,max_purchase_date,first_listen_date,last_listen_date
0,40834,2020-07-01,NaT,NaT,2020-07-08,2020-07-08


## Tính số lượt nghe mỗi tháng

In [133]:
cnt_df = listening_detail_df[["User ID","listen_month_year"]]
h = pd.crosstab(index=cnt_df["User ID"],columns=cnt_df.listen_month_year,dropna=False)
user_cluster_df=user_cluster_df.join(h,how="left",on="User_ID")
month_bins = np.sort(cnt_df.listen_month_year.unique())

user_cluster_df.head(3)

,User_ID,signup_date,change_status_date,max_purchase_date,first_listen_date,last_listen_date,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020
0,40834,2020-07-01,NaT,NaT,2020-07-08,2020-07-08,2.0,0.0,0.0,0.0,0.0,0.0
1,40835,2020-07-01,NaT,NaT,2020-09-02,2020-10-07,0.0,0.0,4.0,1.0,0.0,0.0
2,40839,2020-07-01,NaT,NaT,2020-07-21,2020-10-11,28.0,24.0,3.0,5.0,0.0,0.0


In [132]:
# user_cluster_df=user_cluster_df.drop(columns = month_bins, axis=1)

In [140]:
# month_bins=np.sort(cnt_df.listen_month_year.unique())

In [141]:
user_cluster_df[month_bins].describe()

,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020
count,27183.000000,27183.000000,27183.000000,27183.000000,27183.000000,27183.000000
mean,0.609683,2.027333,2.876504,4.589560,7.591473,9.899827
std,6.523314,11.485216,14.931286,19.076747,24.720837,28.030310
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000
max,368.000000,481.000000,515.000000,462.000000,841.000000,627.000000


In [146]:
## Change Status
# user_cluster_df["min_purchase_date"] = user_cluster_df.change_status_date
# user_cluster_df.change_status_date = user_cluster_df["min_purchase_date"].map(lambda x: x.strftime('%m/%Y'))
user_cluster_df.change_status_date = user_cluster_df["min_purchase_date"].dt.strftime('%m/%Y')
user_cluster_df.tail(5)

In [149]:
user_cluster_df.tail(3)

,User_ID,signup_date,change_status_date,max_purchase_date,first_listen_date,last_listen_date,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,min_purchase_date
27455,106971,2020-12-31,12/2020,2020-12-31,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-31
27456,106974,2020-12-31,12/2020,2020-12-31,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-31
27457,106981,2020-12-31,12/2020,2020-12-31,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-31
